In [1]:
import gym
import numpy as np
import torch as th
import inventory_model
import pandas as pd
from evaluate import *
from ppo_evaluate import ppo_evaluate
import matplotlib.pyplot as plt


from stable_baselines3 import PPO
from stable_baselines3.ppo import MlpPolicy
from stable_baselines3.common.env_util import make_vec_env

In [2]:
import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
#wrapper for cont env with PPO, plot result for several steps
def ppo_eval_interval(p, L, t_t, n_iter, n_step, gae, learning_rate=0.0003):
    ContCONFIG = {'h': 1, 'p': p, 'L': L, 'lambda': 1, 'action': 400}
    PolicyCONFIG = dict(activation_fn=th.nn.Tanh,
                     net_arch=[dict(pi=[64,64], vf=[64, 64])])
    cont_env = make_vec_env('inventory_cont_disc_action_config_fix_model-v0', n_envs=4, env_kwargs=ContCONFIG)
    print("Running PPO w/: p=", p, ", L=",L)
    cont_model = PPO(MlpPolicy, cont_env, verbose=1, gamma = 1, gae_lambda=gae, n_epochs = 16,
                     learning_rate = learning_rate,use_sde = False, n_steps = n_step, policy_kwargs = PolicyCONFIG)
    env_eval = make_vec_env('inventory_cont_disc_action_config_fix_model-v0', n_envs=1, env_kwargs=ContCONFIG)
    timesteps = 0
    numiter = n_iter#test
    res_mean_arr = []
    res_std_arr = []

    while(timesteps <= t_t):

        cont_model.learn(total_timesteps=2*4*n_step-1)#each iteration has 8192 timesteps with n_env=4
        timesteps = timesteps + 2*4*n_step

        res_mean, res_std = ppo_evaluate(cont_model, env_eval, numiter)
        res_mean_arr.append(-res_mean)
        res_std_arr.append(res_std)
        
        if -res_mean == min(res_mean_arr):
            cont_model.save("ppo_min_disc_model_"+str(p)+"_"+str(L))
        
#         print(res_mean_arr)
    
    plt.plot(res_mean_arr)
    plt.xlabel("Iteration")
    plt.ylabel("Average cost")
    plt.title("L="+str(L)+", p="+str(p))
    plt.yscale('log')
    plt.show()
    
    min_model = PPO.load("ppo_min_disc_model_"+str(p)+"_"+str(L))
    mean_min, std_min = ppo_evaluate(min_model, env_eval, 50000)
    print("p="+str(p)+"， L="+str(L)+": mean "+str(-mean_min)+", std_dev: "+str(std_min))
    
    return res_mean_arr, res_std_arr, mean_min, std_min

In [ ]:
# listp = [0.25,1,4,9,39,99]
listp = [4,9,39,99]
# listL = [1,4,70,100]
# listp = [99]
listL = [1]
# t_t = 100000
n_iter = 500
gae_lambda = 0.95
# n_step = 8192
learning_rate = 0.0003
ppo_res = pd.DataFrame(columns = ['p','L','res_mean', 'res_std'])

for p in listp:
    for L in listL:
        n_step = 32*L 
        t_t = 1200*4*n_step
        res_mean, res_std, mean_min, std_min = ppo_eval_interval(p,L,t_t, n_iter, n_step, gae_lambda, learning_rate)
        ppo_res = ppo_res.append({'p': p, 'L':L, 'res_mean':-mean_min, 'res_std': std_min}, ignore_index=True)

Running PPO w/: p= 4 , L= 1
Using cpu device
-----------------------------
| time/              |      |
|    fps             | 6493 |
|    iterations      | 1    |
|    time_elapsed    | 0    |
|    total_timesteps | 128  |
-----------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 1802          |
|    iterations           | 2             |
|    time_elapsed         | 0             |
|    total_timesteps      | 256           |
| train/                  |               |
|    approx_kl            | 0.00019500032 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -5.99         |
|    explained_variance   | 0.00131       |
|    learning_rate        | 0.0003        |
|    loss                 | 3.31e+04      |
|    n_updates            | 16            |
|    policy_gradient_loss | -0.0194       |
|    value_loss           | 6.35e+04     

-------------------------------------------
| time/                   |               |
|    fps                  | 1892          |
|    iterations           | 2             |
|    time_elapsed         | 0             |
|    total_timesteps      | 256           |
| train/                  |               |
|    approx_kl            | -0.0022371933 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -5.99         |
|    explained_variance   | -0.00241      |
|    learning_rate        | 0.0003        |
|    loss                 | 9.2e+03       |
|    n_updates            | 176           |
|    policy_gradient_loss | -0.0243       |
|    value_loss           | 1.83e+04      |
-------------------------------------------
mean:  -244.10943498470783
standard deviation: 20.883430691327643
------------------------------------------
| time/                   |              |
|    fps                  | 7209         |
|    iteratio

mean:  -246.67077899101076
standard deviation: 23.9783492886509
-------------------------------------------
| time/                   |               |
|    fps                  | 7554          |
|    iterations           | 1             |
|    time_elapsed         | 0             |
|    total_timesteps      | 128           |
| train/                  |               |
|    approx_kl            | -0.0008751154 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -5.98         |
|    explained_variance   | 9.54e-07      |
|    learning_rate        | 0.0003        |
|    loss                 | 1.69e+05      |
|    n_updates            | 352           |
|    policy_gradient_loss | -0.00802      |
|    value_loss           | 3.46e+05      |
-------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 1678          |
|    iterati

-------------------------------------------
| time/                   |               |
|    fps                  | 1727          |
|    iterations           | 2             |
|    time_elapsed         | 0             |
|    total_timesteps      | 256           |
| train/                  |               |
|    approx_kl            | -0.0032295175 |
|    clip_fraction        | 0.000977      |
|    clip_range           | 0.2           |
|    entropy_loss         | -5.98         |
|    explained_variance   | 0.00224       |
|    learning_rate        | 0.0003        |
|    loss                 | 2.63e+04      |
|    n_updates            | 528           |
|    policy_gradient_loss | -0.0177       |
|    value_loss           | 4.98e+04      |
-------------------------------------------
mean:  -241.5909350435674
standard deviation: 6.102009588317591
-------------------------------------------
| time/                   |               |
|    fps                  | 6589          |
|    iterati

-------------------------------------------
| time/                   |               |
|    fps                  | 1919          |
|    iterations           | 2             |
|    time_elapsed         | 0             |
|    total_timesteps      | 256           |
| train/                  |               |
|    approx_kl            | -0.0028403737 |
|    clip_fraction        | 0             |
|    clip_range           | 0.2           |
|    entropy_loss         | -5.98         |
|    explained_variance   | 0.00149       |
|    learning_rate        | 0.0003        |
|    loss                 | 2.52e+04      |
|    n_updates            | 688           |
|    policy_gradient_loss | -0.0177       |
|    value_loss           | 4.93e+04      |
-------------------------------------------
mean:  -237.49250622695982
standard deviation: 19.366080094555432
------------------------------------------
| time/                   |              |
|    fps                  | 7395         |
|    iteratio

...

-------------------------------------------
| time/                   |               |
|    fps                  | 1800          |
|    iterations           | 2             |
|    time_elapsed         | 0             |
|    total_timesteps      | 256           |
| train/                  |               |
|    approx_kl            | -0.0023752134 |
|    clip_fraction        | 0.0527        |
|    clip_range           | 0.2           |
|    entropy_loss         | -2.42         |
|    explained_variance   | -1.19e-06     |
|    learning_rate        | 0.0003        |
|    loss                 | 337           |
|    n_updates            | 7984          |
|    policy_gradient_loss | -0.0241       |
|    value_loss           | 626           |
-------------------------------------------
mean:  -2.251653433894669
standard deviation: 0.03913379555478033
------------------------------------------
| time/                   |              |
|    fps                  | 7120         |
|    iteratio

mean:  -2.286216167667945
standard deviation: 0.11049154349059262
-------------------------------------------
| time/                   |               |
|    fps                  | 7372          |
|    iterations           | 1             |
|    time_elapsed         | 0             |
|    total_timesteps      | 128           |
| train/                  |               |
|    approx_kl            | 0.00053903833 |
|    clip_fraction        | 0.0288        |
|    clip_range           | 0.2           |
|    entropy_loss         | -2.34         |
|    explained_variance   | -9.54e-07     |
|    learning_rate        | 0.0003        |
|    loss                 | 318           |
|    n_updates            | 8160          |
|    policy_gradient_loss | -0.0218       |
|    value_loss           | 639           |
-------------------------------------------
----------------------------------------
| time/                   |            |
|    fps                  | 1708       |
|    iterations    

---------------------------------------
| time/                   |           |
|    fps                  | 1738      |
|    iterations           | 2         |
|    time_elapsed         | 0         |
|    total_timesteps      | 256       |
| train/                  |           |
|    approx_kl            | 0.0221385 |
|    clip_fraction        | 0.112     |
|    clip_range           | 0.2       |
|    entropy_loss         | -2.91     |
|    explained_variance   | -1.07e-06 |
|    learning_rate        | 0.0003    |
|    loss                 | 345       |
|    n_updates            | 8336      |
|    policy_gradient_loss | -0.0305   |
|    value_loss           | 755       |
---------------------------------------
mean:  -2.2175674328602852
standard deviation: 0.10729214777076843
-----------------------------------------
| time/                   |             |
|    fps                  | 7225        |
|    iterations           | 1           |
|    time_elapsed         | 0           |
|  

mean:  -2.159787892683595
standard deviation: 0.106825756204504
------------------------------------------
| time/                   |              |
|    fps                  | 7204         |
|    iterations           | 1            |
|    time_elapsed         | 0            |
|    total_timesteps      | 128          |
| train/                  |              |
|    approx_kl            | 0.0086155385 |
|    clip_fraction        | 0.0972       |
|    clip_range           | 0.2          |
|    entropy_loss         | -2.83        |
|    explained_variance   | 0            |
|    learning_rate        | 0.0003       |
|    loss                 | 271          |
|    n_updates            | 8512         |
|    policy_gradient_loss | -0.0285      |
|    value_loss           | 540          |
------------------------------------------
-------------------------------------------
| time/                   |               |
|    fps                  | 1770          |
|    iterations           | 2 

----------------------------------------
| time/                   |            |
|    fps                  | 1772       |
|    iterations           | 2          |
|    time_elapsed         | 0          |
|    total_timesteps      | 256        |
| train/                  |            |
|    approx_kl            | 0.02436984 |
|    clip_fraction        | 0.0757     |
|    clip_range           | 0.2        |
|    entropy_loss         | -2.73      |
|    explained_variance   | -2.98e-06  |
|    learning_rate        | 0.0003     |
|    loss                 | 310        |
|    n_updates            | 8688       |
|    policy_gradient_loss | -0.0274    |
|    value_loss           | 624        |
----------------------------------------
mean:  -2.255357710182224
standard deviation: 0.0924128894055764
----------------------------------------
| time/                   |            |
|    fps                  | 7313       |
|    iterations           | 1          |
|    time_elapsed         | 0    

In [ ]:
ppo_res

In [ ]:
plt.plot(res_mean)

## Below is obsolete.